In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 7.2 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=e27765cbe9a33831efd4f7bd03c3d85dbde70926c0a66d3d453c393fe7652dd3
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [3]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from lifelines.utils import concordance_index
from sklearn.model_selection import StratifiedKFold



In [4]:
BASE = "/content/drive/MyDrive/permanent_data_folder/TCGA_BRCA_data"

# embeddings
X_img = np.stack([
    np.load(os.path.join(BASE, "image_embeddings", f))
    for f in sorted(os.listdir(os.path.join(BASE, "image_embeddings")))
])

X_rna = np.load(os.path.join(BASE, "rnaseq_embeddings", "X_rnaseq_pca.npy"))

# survival labels
cohort = pd.read_csv(os.path.join(BASE, "cohort_image_valid.csv"))

time = cohort["time"].values.astype(float)
event = cohort["event"].values.astype(int)

print(X_img.shape, X_rna.shape, time.shape)


(193, 2048) (193, 128) (193,)


In [5]:
img_dir = os.path.join(BASE, "image_embeddings")

X_img = []
for pid in cohort["patient_id"]:
    path = os.path.join(img_dir, f"{pid}.npy")
    emb = np.load(path)

    if np.isnan(emb).any():
        raise ValueError(f"NaNs in image embedding for {pid}")

    X_img.append(emb)

X_img = np.stack(X_img)


In [6]:
X_rna = np.nan_to_num(X_rna, nan=0.0, posinf=0.0, neginf=0.0)


In [7]:
idx = np.arange(len(time))

train_idx, val_idx = train_test_split(
    idx, test_size=0.2, stratify=event, random_state=42
)

def split(arr):
    return arr[train_idx], arr[val_idx]

X_img_tr, X_img_va = split(X_img)
X_rna_tr, X_rna_va = split(X_rna)
time_tr, time_va   = split(time)
event_tr, event_va = split(event)


In [8]:
# ---- RNA: absolute hardening ----
X_rna = np.nan_to_num(X_rna, nan=0.0, posinf=0.0, neginf=0.0)

# clip extreme PCA values
X_rna = np.clip(X_rna, -10, 10)

# ---- IMAGE: normalize & clip ----
X_img = np.nan_to_num(X_img, nan=0.0, posinf=0.0, neginf=0.0)

# L2-normalize image embeddings (CRITICAL)
X_img = X_img / (np.linalg.norm(X_img, axis=1, keepdims=True) + 1e-8)

# optional extra safety
X_img = np.clip(X_img, -5, 5)

print("After hardening:")
print("NaNs X_img:", np.isnan(X_img).sum())
print("NaNs X_rna:", np.isnan(X_rna).sum())


After hardening:
NaNs X_img: 0
NaNs X_rna: 0


In [9]:
valid = (~np.isnan(time)) & (~np.isnan(event)) & (time > 0)

print("Patients before:", len(time))
print("Patients after :", valid.sum())

X_img = X_img[valid]
X_rna = X_rna[valid]
time  = time[valid]
event = event[valid]


Patients before: 193
Patients after : 193


In [10]:
import pandas as pd

COHORT_PATH = "/content/drive/MyDrive/permanent_data_folder/TCGA_BRCA_data/cohort_image_valid.csv"
cohort_df = pd.read_csv(COHORT_PATH)

print("Loaded cohort:", cohort_df.shape)
cohort_df.head()


Loaded cohort: (193, 5)


,patient_id,time,event,rnaseq_path,slide_path
0,TCGA-A8-A07E,608.0,0,/content/drive/MyDrive/permanent_data_folder/T...,/content/drive/MyDrive/permanent_data_folder/T...
1,TCGA-D8-A146,643.0,0,/content/drive/MyDrive/permanent_data_folder/T...,/content/drive/MyDrive/permanent_data_folder/T...
2,TCGA-A8-A09K,912.0,0,/content/drive/MyDrive/permanent_data_folder/T...,/content/drive/MyDrive/permanent_data_folder/T...
3,TCGA-A2-A0YT,723.0,1,/content/drive/MyDrive/permanent_data_folder/T...,/content/drive/MyDrive/permanent_data_folder/T...
4,TCGA-C8-A130,370.0,0,/content/drive/MyDrive/permanent_data_folder/T...,/content/drive/MyDrive/permanent_data_folder/T...


In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# ---- TRAIN tensors ----
X_img_tr_t = torch.tensor(X_img_tr, dtype=torch.float32).to(device)
X_rna_tr_t = torch.tensor(X_rna_tr, dtype=torch.float32).to(device)
time_tr_t  = torch.tensor(time_tr,  dtype=torch.float32).to(device)
event_tr_t = torch.tensor(event_tr, dtype=torch.float32).to(device)

# ---- VAL tensors ----
X_img_va_t = torch.tensor(X_img_va, dtype=torch.float32).to(device)
X_rna_va_t = torch.tensor(X_rna_va, dtype=torch.float32).to(device)
time_va_t  = torch.tensor(time_va,  dtype=torch.float32).to(device)
event_va_t = torch.tensor(event_va, dtype=torch.float32).to(device)


In [38]:
class ImageRNAContrastive(nn.Module):
    def __init__(self, img_dim=2048, rna_dim=128, proj_dim=128):
        super().__init__()

        # Image encoder (reuse your logic: token → pooled)
        self.img_proj = nn.Linear(img_dim, proj_dim)
        self.img_attn = nn.MultiheadAttention(proj_dim, 2, batch_first=True)

        # RNA encoder
        self.rna_proj = nn.Linear(rna_dim, proj_dim)

    def forward(self, x_img_tokens, x_rna):
        # x_img_tokens: (B, K, 2048)
        img = self.img_proj(x_img_tokens)
        img, _ = self.img_attn(img, img, img)
        img = img.mean(dim=1)           # (B, D)

        rna = self.rna_proj(x_rna)      # (B, D)

        # L2 normalize (CRITICAL for contrastive)
        img = F.normalize(img, dim=1)
        rna = F.normalize(rna, dim=1)

        return img, rna


In [39]:
def contrastive_loss(img_emb, rna_emb, temperature=0.07):
    """
    img_emb, rna_emb: (B, D)
    """
    logits = img_emb @ rna_emb.T / temperature
    labels = torch.arange(len(img_emb)).to(img_emb.device)

    loss_i2r = F.cross_entropy(logits, labels)
    loss_r2i = F.cross_entropy(logits.T, labels)

    return (loss_i2r + loss_r2i) / 2


In [40]:
EPOCHS = 300
LR = 1e-3

model_contrast = ImageRNAContrastive().to(device)
optimizer = torch.optim.Adam(model_contrast.parameters(), lr=LR)

X_img_t = torch.tensor(X_img_tokens, dtype=torch.float32).to(device)
X_rna_t = torch.tensor(X_rna, dtype=torch.float32).to(device)

for epoch in range(EPOCHS):
    model_contrast.train()

    img_emb, rna_emb = model_contrast(X_img_t, X_rna_t)
    loss = contrastive_loss(img_emb, rna_emb)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1} | Contrastive loss: {loss.item():.4f}")


Epoch 50 | Contrastive loss: 0.0978
Epoch 100 | Contrastive loss: 0.0197
Epoch 150 | Contrastive loss: 0.0119
Epoch 200 | Contrastive loss: 0.0084
Epoch 250 | Contrastive loss: 0.0065
Epoch 300 | Contrastive loss: 0.0052


In [12]:
class ImageGuidedCrossAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            batch_first=True
        )

    def forward(self, img_tok, rna_tok):
        # img_tok: (B, 1, d_model) → QUERY
        # rna_tok: (B, 1, d_model) → KEY, VALUE

        z, attn_weights = self.attn(
            query=img_tok,
            key=rna_tok,
            value=rna_tok
        )

        return z, attn_weights


In [33]:
import torch
import torch.nn as nn

class MultimodalSurvivalTransformer(nn.Module):
    def __init__(
        self,
        img_dim=2048,
        rna_dim=128,
        d_model=256,
        n_heads=4
    ):
        super().__init__()

        self.dropout = nn.Dropout(0.3)


        # -------- Image branch --------
        self.img_proj = nn.Linear(img_dim, d_model)


        self.img_self_attn = nn.MultiheadAttention(
            d_model, n_heads, batch_first=True
        )

        # 🔧 LEVER A: compress image tokens
        self.img_compress = nn.Linear(d_model, d_model)

        # -------- RNA branch --------
        self.rna_proj = nn.Linear(rna_dim, d_model)

        # -------- Cross-attention --------
        self.cross_attn = nn.MultiheadAttention(
            d_model, n_heads, batch_first=True
        )

        # -------- Survival head --------
        self.risk_head = nn.Linear(d_model, 1)

    def forward(self, x_img_tokens, x_rna):
    # 🔒 HARD SAFETY
      if x_img_tokens.dim() == 2:
          x_img_tokens = x_img_tokens.unsqueeze(1)

      if x_rna.dim() == 3:
          x_rna = x_rna.squeeze(1)

      # Project
      img = self.img_proj(x_img_tokens)            # (B,K,D)
      rna = self.rna_proj(x_rna).unsqueeze(1)      # (B,1,D)

      # Image self-attention
      img, _ = self.img_self_attn(img, img, img)

      # Compress image tokens → 1 token
      img = self.img_compress(img).mean(dim=1, keepdim=True)

      # Cross-attention
      z, _ = self.cross_attn(img, rna, rna)
      z = z.squeeze(1)

    # Dropout AFTER z exists
      z = self.dropout(z)

      risk = torch.clamp(
          self.risk_head(z).squeeze(-1),
          -20, 20
      )
      return risk




In [14]:
def load_image_tokens(pid, token_dir, K=8):
    x = np.load(os.path.join(token_dir, f"{pid}.npy"))
    if x.shape[0] < K:
        pad = np.zeros((K - x.shape[0], x.shape[1]))
        x = np.vstack([x, pad])
    return x[:K]


In [15]:
patient_ids = cohort_df["patient_id"].tolist()

TOKEN_DIR = "/content/drive/MyDrive/permanent_data_folder/TCGA_BRCA_data/image_tokens"
X_img_tokens = np.stack([
    load_image_tokens(pid, TOKEN_DIR)
    for pid in patient_ids
])  # (N, K, 2048)


In [16]:
print("Image token tensor:", X_img_tokens.shape)


Image token tensor: (193, 8, 2048)


In [17]:
from sklearn.model_selection import train_test_split

idx = np.arange(len(time))
train_idx, val_idx = train_test_split(
    idx, test_size=0.2, stratify=event, random_state=42
)

X_img_tr = X_img_tokens[train_idx]
X_img_va = X_img_tokens[val_idx]
X_rna_tr = X_rna[train_idx]
time_tr  = time[train_idx]
event_tr = event[train_idx]


In [18]:
def pairwise_rank_loss(risk, time, event):
    loss = 0.0
    count = 0

    for i in range(len(time)):
        if event[i] == 1:
            valid = time > time[i]
            if valid.sum() > 0:
                diff = risk[i] - risk[valid]
                loss += torch.log1p(torch.exp(-diff)).mean()
                count += 1

    return loss / max(count, 1)


In [19]:
def cox_loss_weighted(risk, time, event, alpha=2.0):
    """
    alpha > 1 increases weight of observed events
    """
    risk = risk.view(-1)
    time = time.view(-1)
    event = event.view(-1)

    order = torch.argsort(time, descending=True)
    risk = risk[order]
    event = event[order]

    log_cumsum = torch.logcumsumexp(risk, dim=0)

    weights = torch.ones_like(event)
    weights[event == 1] = alpha

    loss = -torch.sum(weights * event * (risk - log_cumsum))
    return loss / (event.sum() + 1e-8)


In [20]:
def sensitivity_regularization(model, x_img, x_rna, sens, noise=0.1):
    with torch.no_grad():
        risk_orig, _ = model(x_img, x_rna)

    x_rna_pert = x_rna + noise * torch.randn_like(x_rna)
    risk_pert, _ = model(x_img, x_rna_pert)

    delta = torch.abs(risk_pert - risk_orig)

    std = delta.std()
    if std < 1e-4:
        return torch.tensor(0.0, device=delta.device)

    delta = (delta - delta.mean()) / std
    sens = torch.sigmoid(sens)

    return F.mse_loss(sens, delta)


In [21]:
# split
X_img_tr = X_img_tokens[train_idx]
X_img_va = X_img_tokens[val_idx]

X_rna_tr = X_rna[train_idx]
X_rna_va = X_rna[val_idx]

time_tr = time[train_idx]
event_tr = event[train_idx]

X_img_va_t = torch.tensor(X_img_va, dtype=torch.float32).to(device)
X_rna_va_t = torch.tensor(X_rna_va, dtype=torch.float32).to(device)
time_va_t  = torch.tensor(time_va,  dtype=torch.float32).to(device)
event_va_t = torch.tensor(event_va, dtype=torch.float32).to(device)

# tensors
X_img_tr_t = torch.tensor(X_img_tr, dtype=torch.float32).to(device)
X_rna_tr_t = torch.tensor(X_rna_tr, dtype=torch.float32).to(device)
time_tr_t = torch.tensor(time_tr, dtype=torch.float32).to(device)
event_tr_t = torch.tensor(event_tr, dtype=torch.float32).to(device)

# model
model = MultimodalSurvivalTransformer().to(device)

# forward sanity check
r = model(X_img_tr_t, X_rna_tr_t)
print("Risk OK:", r.shape)


Risk OK: torch.Size([154])


In [22]:
print("X_img_tr_t shape:", X_img_tr_t.shape)
print("X_rna_tr_t shape:", X_rna_tr_t.shape)
print("X_img_va_t shape:", X_img_va_t.shape)
print("X_rna_va_t shape:", X_rna_va_t.shape)


X_img_tr_t shape: torch.Size([154, 8, 2048])
X_rna_tr_t shape: torch.Size([154, 128])
X_img_va_t shape: torch.Size([39, 8, 2048])
X_rna_va_t shape: torch.Size([39, 128])


In [23]:
model = MultimodalSurvivalTransformer().to(device)
model.eval()
with torch.no_grad():
    r = model(X_img_tr_t, X_rna_tr_t)


print("risk NaNs:", torch.isnan(r).sum().item())
#print("sens NaNs:", torch.isnan(s).sum().item())
print("risk min/max:", r.min().item(), r.max().item())


risk NaNs: 0
risk min/max: -3.0518147945404053 2.7366559505462646


In [24]:
assert X_img_tr_t.dim() == 3
assert X_img_tr_t.shape[1] == 8
assert X_rna_tr_t.dim() == 2



In [42]:
surv_model = MultimodalSurvivalTransformer(
    d_model=128,
    n_heads=2   # IMPORTANT: reduce heads too
).to(device)


In [43]:
surv_model.img_proj.load_state_dict(
    model_contrast.img_proj.state_dict()
)
surv_model.rna_proj.load_state_dict(
    model_contrast.rna_proj.state_dict()
)


<All keys matched successfully>

In [44]:
for p in surv_model.img_proj.parameters():
    p.requires_grad = False
for p in surv_model.rna_proj.parameters():
    p.requires_grad = False


In [25]:
EPOCHS = 40
LR = 1e-4

model = MultimodalSurvivalTransformer().to(device)
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=LR
)

for epoch in range(EPOCHS):
    model.train()

    risk = model(X_img_tr_t, X_rna_tr_t)

    loss = cox_loss_weighted(
        risk,
        time_tr_t,
        event_tr_t,
        alpha=5.0
    )

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        print(
            f"Epoch {epoch+1:02d} | "
            f"Loss = {loss.item():.4f}"
        )


Epoch 05 | Loss = 16.6633
Epoch 10 | Loss = 13.8726
Epoch 15 | Loss = 11.7095
Epoch 20 | Loss = 9.9427
Epoch 25 | Loss = 8.2205
Epoch 30 | Loss = 6.7092
Epoch 35 | Loss = 5.4093
Epoch 40 | Loss = 4.3821


In [26]:
c_indices = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


for fold, (tr, va) in enumerate(skf.split(X_img_tokens, event), 1):

    print(f"\nFold {fold}")
    print(f"  Train events: {event[tr].sum()} | Val events: {event[va].sum()}")

    # ---------- TRAIN DATA ----------
    X_img_tr_t = torch.tensor(X_img[tr], dtype=torch.float32).to(device)
    X_rna_tr_t = torch.tensor(X_rna[tr], dtype=torch.float32).to(device)
    time_tr_t  = torch.tensor(time[tr], dtype=torch.float32).to(device)
    event_tr_t = torch.tensor(event[tr], dtype=torch.float32).to(device)

    # ---------- VAL DATA ----------
    X_img_va_t = torch.tensor(X_img[va], dtype=torch.float32).to(device)
    X_rna_va_t = torch.tensor(X_rna[va], dtype=torch.float32).to(device)

    time_va  = time[va]
    event_va = event[va]



Fold 1
  Train events: 23 | Val events: 6

Fold 2
  Train events: 23 | Val events: 6

Fold 3
  Train events: 23 | Val events: 6

Fold 4
  Train events: 24 | Val events: 5

Fold 5
  Train events: 23 | Val events: 6


In [27]:
N_FOLDS = 5
EPOCHS = 40
LR = 1e-4

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
c_indices = []


In [34]:
from sklearn.model_selection import StratifiedShuffleSplit
from lifelines.utils import concordance_index
import numpy as np


In [35]:
N_REPEATS = 10
TEST_SIZE = 0.2
EPOCHS = 200
LR = 1e-4

sss = StratifiedShuffleSplit(
    n_splits=N_REPEATS,
    test_size=TEST_SIZE,
    random_state=42
)

c_indices = []


In [45]:
for split_id, (tr, va) in enumerate(sss.split(X_img_tokens, event), 1):
    print(f"\n===== Split {split_id} =====")
    print(
        " Train events:", event[tr].sum(),
        "| Val events:", event[va].sum()
    )

    # ---------- DATA ----------
    X_img_tr_t = torch.tensor(X_img_tokens[tr], dtype=torch.float32).to(device)
    X_rna_tr_t = torch.tensor(X_rna[tr], dtype=torch.float32).to(device)
    time_tr_t  = torch.tensor(time[tr], dtype=torch.float32).to(device)
    event_tr_t = torch.tensor(event[tr], dtype=torch.float32).to(device)

    X_img_va_t = torch.tensor(X_img_tokens[va], dtype=torch.float32).to(device)
    X_rna_va_t = torch.tensor(X_rna[va], dtype=torch.float32).to(device)

    # ---------- MODEL ----------
    model = MultimodalSurvivalTransformer().to(device)
    optimizer = torch.optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=LR
    )

    # ---------- TRAIN ----------
    for epoch in range(EPOCHS):
        model.train()
        risk = model(X_img_tr_t, X_rna_tr_t)

        loss = cox_loss_weighted(
            risk,
            time_tr_t,
            event_tr_t,
            alpha=1.5   # IMPORTANT: lower alpha for stability
        )

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # ---------- VALIDATE ----------
    model.eval()
    with torch.no_grad():
        risk_va = model(X_img_va_t, X_rna_va_t)

    ci = concordance_index(
        time[va],
        -risk_va.cpu().numpy(),
        event[va]
    )

    print(" C-index:", round(ci, 4))
    c_indices.append(ci)



===== Split 1 =====
 Train events: 23 | Val events: 6
 C-index: 0.619

===== Split 2 =====
 Train events: 23 | Val events: 6
 C-index: 0.5238

===== Split 3 =====
 Train events: 23 | Val events: 6
 C-index: 0.3725

===== Split 4 =====
 Train events: 23 | Val events: 6
 C-index: 0.3279

===== Split 5 =====
 Train events: 23 | Val events: 6
 C-index: 0.7423

===== Split 6 =====
 Train events: 23 | Val events: 6
 C-index: 0.3889

===== Split 7 =====
 Train events: 23 | Val events: 6
 C-index: 0.4394

===== Split 8 =====
 Train events: 23 | Val events: 6
 C-index: 0.7348

===== Split 9 =====
 Train events: 23 | Val events: 6
 C-index: 0.4815

===== Split 10 =====
 Train events: 23 | Val events: 6
 C-index: 0.5595


In [46]:
print("\n===== REPEATED STRATIFIED HOLDOUT RESULTS =====")
print("C-indices:", [round(c, 4) for c in c_indices])
print("Mean C-index:", np.mean(c_indices))
print("Std  C-index:", np.std(c_indices))



===== REPEATED STRATIFIED HOLDOUT RESULTS =====
C-indices: [np.float64(0.6429), np.float64(0.4683), np.float64(0.3137), np.float64(0.3115), np.float64(0.5464), np.float64(0.2381), np.float64(0.5), np.float64(0.553), np.float64(0.5309), np.float64(0.7857), np.float64(0.619), np.float64(0.5238), np.float64(0.3725), np.float64(0.3279), np.float64(0.7423), np.float64(0.3889), np.float64(0.4394), np.float64(0.7348), np.float64(0.4815), np.float64(0.5595)]
Mean C-index: 0.5040043724194493
Std  C-index: 0.1486343286016323


In [32]:
print("\n===== 5-FOLD CV RESULTS =====")
print("C-indices:", [round(c, 4) for c in c_indices])
print("Mean C-index:", np.mean(c_indices))
print("Std  C-index:", np.std(c_indices))



===== 5-FOLD CV RESULTS =====
C-indices: [np.float64(0.5), np.float64(0.4471), np.float64(0.663), np.float64(0.4273), np.float64(0.4583), np.float64(0.4167), np.float64(0.6118), np.float64(0.5326), np.float64(0.7273), np.float64(0.4861)]
Mean C-index: 0.5270132268981373
Std  C-index: 0.1007268989687209


In [30]:
model.eval()
with torch.no_grad():
    _, sens_all = model(
        torch.tensor(X_img, dtype=torch.float32).to(device),
        torch.tensor(X_rna, dtype=torch.float32).to(device)
    )

sens_all = sens_all.cpu().numpy()

print("Sensitivity score stats:")
print("mean:", sens_all.mean())
print("std :", sens_all.std())


UnboundLocalError: cannot access local variable 'z' where it is not associated with a value

In [ ]:
with torch.no_grad():
    img_tok = model.img_proj(X_img_va_t)
    rna_tok = model.rna_proj(X_rna_va_t)

    img_norm = img_tok.norm(dim=1)
    rna_norm = rna_tok.norm(dim=1)

    modality_ratio = img_norm / (img_norm + rna_norm + 1e-8)


In [ ]:
import numpy as np

risk_val = r_val.cpu().numpy()
corr = np.corrcoef(modality_ratio.cpu().numpy(), risk_val)[0,1]
print("Modality dominance ↔ risk correlation:", corr)


In [ ]:
import torch.nn as nn

class CoxLinear(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.fc = nn.Linear(in_dim, 1)

    def forward(self, x):
        return self.fc(x).squeeze(-1)


In [ ]:
from lifelines.utils import concordance_index
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cox_rna_scores = []

for fold, (tr, va) in enumerate(skf.split(X_rna, event), 1):
    model = CoxLinear(X_rna.shape[1]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    X_tr = torch.tensor(X_rna[tr], dtype=torch.float32).to(device)
    t_tr = torch.tensor(time[tr], dtype=torch.float32).to(device)
    e_tr = torch.tensor(event[tr], dtype=torch.float32).to(device)

    X_va = torch.tensor(X_rna[va], dtype=torch.float32).to(device)

    for _ in range(200):
        risk = model(X_tr)
        loss = cox_loss_full(risk, t_tr, e_tr)
        opt.zero_grad()
        loss.backward()
        opt.step()

    with torch.no_grad():
        r_va = model(X_va)

    ci = concordance_index(time[va], -r_va.cpu().numpy(), event[va])
    cox_rna_scores.append(ci)

print("Cox RNA CV:", np.mean(cox_rna_scores), "±", np.std(cox_rna_scores))


In [ ]:
from lifelines.utils import concordance_index
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cox_img_scores = []

print("Total patients:", len(X_img))
print("Total events:", event.sum())
print("-" * 50)

for fold, (tr, va) in enumerate(skf.split(X_img_tokens, event), 1):
    print(f"\nFold {fold}")

    print(
        "  Train events:", event[tr].sum(),
        "| Val events:", event[va].sum()
    )

    model = CoxLinear(in_dim=X_img.shape[1]).to(device)  # 2048
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=5e-4,
        weight_decay=1e-4   # regularization = important
    )

    X_tr = torch.tensor(X_img[tr], dtype=torch.float32).to(device)
    t_tr = torch.tensor(time[tr], dtype=torch.float32).to(device)
    e_tr = torch.tensor(event[tr], dtype=torch.float32).to(device)

    X_va = torch.tensor(X_img[va], dtype=torch.float32).to(device)

    # ---- full-batch Cox training ----
    for _ in range(200):
        risk = model(X_tr)
        loss = cox_loss_full(risk, t_tr, e_tr)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # ---- validation ----
    with torch.no_grad():
        r_va = model(X_va)

    ci = concordance_index(
        time[va],
        -r_va.cpu().numpy(),
        event[va]
    )

    print("  C-index:", round(ci, 4))
    cox_img_scores.append(ci)

print("\n===== IMAGE-ONLY COX RESULTS =====")
print("C-indices:", [round(c, 4) for c in cox_img_scores])
print("Mean C-index:", np.mean(cox_img_scores))
print("Std  C-index:", np.std(cox_img_scores))


In [ ]:
class MLPFusion(nn.Module):
    def __init__(self, img_dim, rna_dim, hidden=256):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(img_dim + rna_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_img, x_rna):
        x = torch.cat([x_img, x_rna], dim=1)
        return self.net(x).squeeze(-1)


In [ ]:
mlp_scores = []

for fold, (tr, va) in enumerate(skf.split(X_img, event), 1):
    model = MLPFusion(X_img.shape[1], X_rna.shape[1]).to(device)
    opt = torch.optim.Adam(model.parameters(), lr=1e-3)

    Xi_tr = torch.tensor(X_img[tr], dtype=torch.float32).to(device)
    Xr_tr = torch.tensor(X_rna[tr], dtype=torch.float32).to(device)
    t_tr  = torch.tensor(time[tr], dtype=torch.float32).to(device)
    e_tr  = torch.tensor(event[tr], dtype=torch.float32).to(device)

    Xi_va = torch.tensor(X_img[va], dtype=torch.float32).to(device)
    Xr_va = torch.tensor(X_rna[va], dtype=torch.float32).to(device)

    for _ in range(200):
        risk = model(Xi_tr, Xr_tr)
        loss = cox_loss_full(risk, t_tr, e_tr)
        opt.zero_grad()
        loss.backward()
        opt.step()

    with torch.no_grad():
        r_va = model(Xi_va, Xr_va)

    ci = concordance_index(time[va], -r_va.cpu().numpy(), event[va])
    mlp_scores.append(ci)

print("MLP Fusion CV:", np.mean(mlp_scores), "±", np.std(mlp_scores))
